<a href="https://colab.research.google.com/github/ErdemKilci/ReservoirWind/blob/main/reservoir_wind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas matplotlib openpyxl


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# 1. Angi filbanen for dataene
data_folder = "Data"  # Mappen hvor Excel-filene ligger

# 2. Finn alle Excel-filene i mappen
excel_files = [f for f in os.listdir(data_folder) if f.endswith('.xlsx')]

# 3. Les inn og kombiner alle filene til én DataFrame
all_data = pd.DataFrame()

for file in excel_files:
    file_path = os.path.join(data_folder, file)
    # Les inn filen (forutsetter at alle filer har samme kolonneformat)
    data = pd.read_excel(file_path)

    # Legg til dataen i hoved-DataFrame
    all_data = pd.concat([all_data, data], ignore_index=True)

# 4. Utforsk dataene
print("Data oversikt:")
print(all_data.info())

print("\nEksempeldata:")
print(all_data.head())

FileNotFoundError: [Errno 2] No such file or directory: 'Data'

In [ ]:
# Rensing av data og fjern tomme rad
all_data = all_data.dropna(subset=['Time', 'Forecast Load', 'Actual Load'])
print("Data oversikt:")
print(all_data.info())

Data oversikt:
<class 'pandas.core.frame.DataFrame'>
Index: 88065 entries, 2 to 108485
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Time           88065 non-null  object
 1   Forecast Load  88065 non-null  object
 2   Actual Load    88065 non-null  object
dtypes: object(3)
memory usage: 2.7+ MB
None


In [ ]:
print("\nEksempeldata:")
print(all_data.head())


Eksempeldata:
            Time Forecast Load Actual Load
2  00:00 - 01:00         17872       18350
3  01:00 - 02:00         18118       18364
4  02:00 - 03:00         17993       18224
5  03:00 - 04:00         17851       18030
6  04:00 - 05:00         17827       18009


In [ ]:
# Konverter tid til datetime-format
all_data['Time'] = pd.to_datetime(all_data['Time'], errors='coerce')

<ipython-input-8-bba5d406eede>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_data['Time'] = pd.to_datetime(all_data['Time'], errors='coerce')
